### 3. Model selection

In [1]:
def make_datasets(training_data_path: str, image_dim: int, batch_size: int=16):

    training_dataset=tf.keras.utils.image_dataset_from_directory(
        training_data_path,
        validation_split=0.2,
        subset='training',
        seed=315,
        image_size=(image_dim, image_dim),
        batch_size=batch_size
    ).repeat()

    validation_dataset=tf.keras.utils.image_dataset_from_directory(
        training_data_path,
        validation_split=0.2,
        subset='validation',
        seed=315,
        image_size=(image_dim, image_dim),
        batch_size=batch_size
    ).repeat()



    return training_dataset, validation_dataset,


training_dataset, validation_dataset=make_datasets(training_data_path, 128)

NameError: name 'training_data_path' is not defined

### 3.1 Model Definition

In [ ]:
def compile_model(image_dim, learning_rate):

    initializer=tf.keras.initializers.GlorotUniform(seed=315)

    model=Sequential([
        layers.Input((image_dim, image_dim, 3)),
        layers.Rescaling(1./255),
        layers.Conv2D(16, 3, padding='same', activation='relu', kernel_initializer=initializer),
        layers.MaxPooling2D(),
        layers.Conv2D(32, 3, padding='same', activation='relu', kernel_initializer=initializer),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, padding='same', activation='relu', kernel_initializer=initializer),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation='relu', kernel_initializer=initializer),
        layers.Dense(1, activation='sigmoid', kernel_initializer=initializer)
    ])

    optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['binary_accuracy'])

    return model

model=compile_model(128, 0.001)
model.summary()

### 3.2. Model training

In [ ]:
training_results=model.fit(
  training_dataset,
  validation_data=validation_dataset,
  epochs=10,
  steps_per_epoch=5,
  validation_steps=5
)

In [ ]:
print(training_results.history.keys())

In [ ]:
# Take a look at what information 'training_results' contains. Plot the training and validation accuracy (and binary cross-entropy if you like) over the training epoch. Is the model learning? If not, what do you think is wrong?


import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(training_results.history['binary_accuracy'], label='Train Accuracy')
plt.plot(training_results.history['val_binary_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(training_results.history['loss'], label='Train Loss')
plt.plot(training_results.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')

plt.tight_layout()
plt.show()

### 3.3 Model Optimization

In [ ]:
learning_rates = [0.1, 0.01, 0.001]
batch_size = [16, 32, 64]
results = []

training_data_path = '../data/data/train'

for lr in learning_rates:
    for bs in batch_size:
        
        training_dataset, validation_dataset = make_datasets(training_data_path, image_dim=128, batch_size=bs)

       
        model = compile_model(image_dim=128, learning_rate=lr)

       
        result = model.fit(training_dataset, validation_data=validation_dataset, epochs=10, steps_per_epoch=5, validation_steps=5)

       
        results.append((lr, bs, result))

### 4 Model Evaluation 

In [ ]:
for lr, bs, result in results:
    print(f'Learning Rate: {lr}, Batch Size: {bs}')
    print(f'Train Accuracy: {result.history["binary_accuracy"][-1]}')
    print(f'Validation Accuracy: {result.history["val_binary_accuracy"][-1]}')